In [2]:
import pandas as pd 
import numpy as np  

In [3]:

train_path = 'LIAR/train.tsv'
train_data = pd.read_csv(train_path,sep='\t', header=None, names=["id", "label", "statement", "subject(s)", "speaker","speaker's job title", "state info", "party affiliation", "barely true counts", "false counts","half true counts", "mostly true counts", "pants on fire counts", "context"])

train_data['label'] = train_data['label'].replace(['pants-fire', 'barely-true','false'], 0)
train_data['label'] = train_data['label'].replace(['half-true', 'mostly-true','true'], 1)

In [4]:
from nltk.sentiment import SentimentIntensityAnalyzer
# import nltk 
# nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
def analyze_sentiment(text):
    if isinstance(text, str) == False and text != 'NaN':
        return None
    words = word_tokenize(text)
   
    # filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(words)

    sentiment_scores = sid.polarity_scores(filtered_text) 
    return sentiment_scores['compound'] +1

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
stop_words = set(stopwords.words('english'))
train_data['sentiment'] = train_data['statement'].apply(analyze_sentiment)
train_features = train_data[['sentiment', 'statement']]
y_train = train_data['label']
train_features.head()


,sentiment,statement
0,1.2500,Says the Annies List political group supports ...
1,1.3612,When did the decline of coal start? It started...
2,1.3182,"Hillary Clinton agrees with John McCain ""by vo..."
3,1.7579,Health care reform legislation is likely to ma...
4,1.0000,The economic turnaround started at the end of ...


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,  pipeline
from scipy.special import softmax
roberta_link = 'jy46604790/Fake-News-Bert-Detect'
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_link)
new_tokenizer = AutoTokenizer.from_pretrained(roberta_link)

ModuleNotFoundError: No module named 'transformers'

In [ ]:
for statement in train_features['statement'][:10]:
    encoded_statement = new_tokenizer(statement,return_tensors="pt")
    output = model(**encoded_statement)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    for i in range (0, len(labels)):
        print(f"{labels[i]}: {(scores[i] * 100)}%")